In [1]:
import pandas as pd 
import numpy  as np 

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

In [108]:
df = pd.read_csv("output/final_processed_articles.csv")



In [109]:
df['words'] = df['words'].apply(lambda x: eval(x) if isinstance(x, str) else x)

df['text_embeddings'] = df['text_embeddings'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [110]:
text_embeddings = list(df["text_embeddings"].values)


In [111]:
mlb = MultiLabelBinarizer()

one_hot_encoded = mlb.fit_transform(df['words'])

cosine_sim_matrix = cosine_similarity(one_hot_encoded)

In [112]:
def fetch_nearest_neighbors(row_index,cosine_sim_matrix, s):

    similarity_scores = cosine_sim_matrix[row_index]
    
    sorted_indices = similarity_scores.argsort()[::-1]
    

    sorted_indices = sorted_indices[1:]
    

    nearest_neighbors_indices = sorted_indices[:s]
    
    return nearest_neighbors_indices




In [113]:
def find_cosine_similar_embeddings(query_embedding, embedding_list, s):
    
    similarities = cosine_similarity([query_embedding], embedding_list)[0]

    sorted_indices = np.argsort(similarities)[::-1]

    sorted_indices = sorted_indices[1:]
    
    most_similar_indices = sorted_indices[:s]
    
    return most_similar_indices

In [114]:
def fetch_semantically_divergent(query_embedding, embedding_list, d):

    similarities = cosine_similarity([query_embedding], embedding_list)[0]

    # similarities = [np.abs(similarity) for similarity in similarities]

    sorted_indices = np.argsort(similarities)
    
    most_similar_indices = sorted_indices[:d]
    
    return most_similar_indices



In [115]:
query_index = 47

### Nearest Neighbour + Divergent

In [116]:
similarity_num, divergent_num = 8,4 

In [117]:

query_embedding  = text_embeddings[query_index]

nearest_neighbors_indices = fetch_nearest_neighbors(query_index, cosine_sim_matrix, s=similarity_num)

similar_pre_picked_embeddings =  [text_embeddings[similar_index] for similar_index in nearest_neighbors_indices]

divergent_indices = fetch_semantically_divergent(query_embedding,similar_pre_picked_embeddings,d=divergent_num )

query_headline = df.iloc[query_index]["headline"] 

nearest_headlines = df.iloc[nearest_neighbors_indices]["headline"].values 

divergent_headlines = df.iloc[divergent_indices]["headline"].values

In [118]:
print(query_headline, "\n\n\n","Similar Headlines")
print(nearest_headlines,"\n\n\n","Divergent Headlines")
print(divergent_headlines)

Biden and Trump trade barbs at rival rallies in Georgia 


 Similar Headlines
["Here are Georgia's 2024 presidential primary results"
 "Biden and Trump could clinch presidential nominations after tonight's elections"
 "It's not just inner-ring suburbs. Trump has also underperformed in key exurban counties during the GOP primaries."
 "A Trump vs Biden rematch is on, and it's a showdown most Americans didn't want"
 "Mark Cuban says he's not voting for Donald Trump because he doesn't want 'a snake oil salesperson as President'"
 'Biden is using his cash advantage to address vulnerabilities against Trump'
 "Here are Washington's 2024 presidential primary results"
 'How to watch President Joe Biden’s State of the Union address'] 


 Divergent Headlines
['Biggest US election day yet as 15 states vote on Super Tuesday'
 "Key takeaways from Super Tuesday's results so far"
 'Panorama - Trump: The Sequel?'
 'Trump pushes legal challenges in two cases']


### Cosine + Divergent

In [119]:
similarity_num, divergent_num = 8,4 

In [120]:

query_embedding  = text_embeddings[query_index]

nearest_neighbors_indices = find_cosine_similar_embeddings(query_embedding, text_embeddings, s=similarity_num)

similar_pre_picked_embeddings =  [text_embeddings[similar_index] for similar_index in nearest_neighbors_indices]

divergent_indices = fetch_semantically_divergent(query_embedding,similar_pre_picked_embeddings,d =divergent_num )

query_headline = df.iloc[query_index]["headline"] 

nearest_headlines = df.iloc[nearest_neighbors_indices]["headline"].values 

divergent_headlines = df.iloc[divergent_indices]["headline"].values

In [121]:


print(query_headline, "\n\n\n","Similar Headlines")
print(nearest_headlines,"\n\n\n","Divergent Headlines")
print(divergent_headlines)



Biden and Trump trade barbs at rival rallies in Georgia 


 Similar Headlines
["Biden and Trump could clinch presidential nominations after tonight's elections"
 "This is happening: It's Trump versus Biden, and Trump has the upper hand now"
 'Trump again mixed up Obama and Biden during Virginia rally speech'
 "A Trump vs Biden rematch is on, and it's a showdown most Americans didn't want"
 'Biggest US election day yet as 15 states vote on Super Tuesday'
 "Putin said Trump wasn't happy with him in 2020 because he thought the Russian leader wanted 'Sleepy Joe to win'"
 'The ultimate guide to Super Tuesday'
 "It's not just inner-ring suburbs. Trump has also underperformed in key exurban counties during the GOP primaries."] 


 Divergent Headlines
['Biggest US election day yet as 15 states vote on Super Tuesday'
 "Key takeaways from Super Tuesday's results so far"
 "Biden and Trump 'clinch their party nominations'"
 'Trump pushes legal challenges in two cases']
